<a href="https://colab.research.google.com/github/mathubhalan/BERT-NLP/blob/master/BERT_Sentimentanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install transformers

     |████████████████████████████████| 890kB 2.8MB/s 
     |████████████████████████████████| 1.1MB 16.0MB/s 
     |████████████████████████████████| 890kB 22.7MB/s 
     |████████████████████████████████| 3.0MB 25.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=55a35a99f5ab84b2a0f8c1fb851854f4e573519b9381d06638867cd7c806ec6b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


# Exploratory Data Analysis and Preprocessing


In [1]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [4]:
df= pd.read_csv("Train.csv")
df.set_index('Text_ID', inplace=True)

In [5]:
df.head()

,Product_Description,Product_Type,Sentiment
Text_ID,,,
3057,The Web DesignerÛªs Guide to iOS (and Android...,9,2
6254,RT @mention Line for iPad 2 is longer today th...,9,2
8212,Crazy that Apple is opening a temporary store ...,9,2
4422,The lesson from Google One Pass: In this digit...,9,2
5526,RT @mention At the panel: &quot;Your mom has a...,9,2


In [6]:
df.Sentiment.value_counts()

2    3765
3    2089
1     399
0     111
Name: Sentiment, dtype: int64

Class - Represents various sentiments
0. Cannot Say
1. Negative
2. Positive
3. No Sentiment

# Training/Validation Split

In [7]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.Sentiment.values, 
                                                  test_size=0.15, 
                                                  random_state=17, 
                                                  stratify=df.Sentiment.values)

In [14]:
y_train

array([3, 2, 2, ..., 2, 3, 0])

Check to validate if we have equal data in all classes


In [12]:
df['data_type'] = ['not_set']*df.shape[0]

In [15]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [17]:
df.head()

,Product_Description,Product_Type,Sentiment,data_type
Text_ID,,,,
3057,The Web DesignerÛªs Guide to iOS (and Android...,9,2,train
6254,RT @mention Line for iPad 2 is longer today th...,9,2,train
8212,Crazy that Apple is opening a temporary store ...,9,2,train
4422,The lesson from Google One Pass: In this digit...,9,2,train
5526,RT @mention At the panel: &quot;Your mom has a...,9,2,val


In [16]:
df.columns

Index(['Product_Description', 'Product_Type', 'Sentiment', 'data_type'], dtype='object')

In [18]:
df.groupby(['Sentiment', 'data_type']).count()

Product_Description  Product_Type
Sentiment data_type                                   
0         train                       94            94
          val                         17            17
1         train                      339           339
          val                         60            60
2         train                     3200          3200
          val                        565           565
3         train                     1776          1776
          val                        313           313

In [27]:
df[df.data_type=='train'][['Product_Description','Product_Type']].values

array([['The Web Designer\x89Ûªs Guide to iOS (and Android) Apps, today @mention 10 a.m! {link} #sxsw',
        9],
       ['RT @mention Line for iPad 2 is longer today than yesterday. #SXSW  // are you getting in line again today just for fun?',
        9],
       ['Crazy that Apple is opening a temporary store in Austin tomorrow to handle the rabid #sxsw eye pad too seekers.',
        9],
       ...,
       ['SXSW iPhone app is awesome, but iPad app crashes every time. #sxsw',
        7],
       ['RT @mention RT @mention Download 20+ free tracks from @mention Music Sampler @mention including @glove! {link} #SXSW',
        9],
       ['#Google launching a &quot;major&quot; new social network at #sxsw ... Wonder what that can be...',
        9]], dtype=object)

# Loading Tokenizer and Encoding our Data

In [21]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [29]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [30]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].Product_Description.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [31]:
encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].Product_Description.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [179]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].Sentiment.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].Sentiment.values)

In [180]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [37]:
print (f"shape of train is :{len(dataset_train)}")
print (f"shape of val is :{len(dataset_val)}")

shape of train is :5409
shape of val is :955


# Setting up BERT Pretrained Model

In [38]:
from transformers import BertForSequenceClassification

In [40]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=4,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

# Creating Data Loaders

In [41]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [181]:
batch_size = 32

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

# Setting Up Optimiser and Scheduler

In [43]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [44]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [45]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

# Defining our Performance Metrics

In [46]:
import numpy as np
from sklearn.metrics import f1_score

In [48]:
label_dict ={'Cannot Say':0,'Negative':1,'Postive':2,'No Sentiment':3}
label_dict

{'Cannot Say': 0, 'Negative': 1, 'No Sentiment': 3, 'Postive': 2}

In [47]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [49]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

# Creating our Training Loop

In [51]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [52]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [53]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [54]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.9126953619367936
Validation loss: 0.7693215231100718
F1 Score (Weighted): 0.6450803313550091



Epoch 2
Training loss: 0.7335628567373051
Validation loss: 0.7089532206455866
F1 Score (Weighted): 0.6777720180395003



Epoch 3
Training loss: 0.6451701930340599
Validation loss: 0.7013080259164174
F1 Score (Weighted): 0.6845341987651123



Epoch 4
Training loss: 0.5432991111979765
Validation loss: 0.6962456574042638
F1 Score (Weighted): 0.6934673594171611



Epoch 5
Training loss: 0.47098755415748145
Validation loss: 0.7617089341084162
F1 Score (Weighted): 0.7072692078726689



Epoch 6
Training loss: 0.4171408704975072
Validation loss: 0.764098396897316
F1 Score (Weighted): 0.7093056819730302



Epoch 7
Training loss: 0.37025460618383743
Validation loss: 0.7992221822341283
F1 Score (Weighted): 0.7053686908421115



Epoch 8
Training loss: 0.34176409893176135
Validation loss: 0.8283120135466258
F1 Score (Weighted): 0.6971402216672844



Epoch 9
Training loss: 0.3051821860758697
Validation loss: 0.8656876226266225
F1 Score (Weighted): 0.7010128603007236



Epoch 10
Training loss: 0.2961352830643163
Validation loss: 0.8643942276636759
F1 Score (Weighted): 0.7045835792030342



In [55]:
_, predictions, true_vals = evaluate(dataloader_validation)


In [57]:
predictions

array([[-1.61608   , -1.8265791 ,  3.5225694 , -0.646289  ],
       [-1.7421474 , -2.3777046 ,  3.785414  , -0.6429743 ],
       [-1.5066649 , -1.8749869 , -0.0077248 ,  3.7427683 ],
       ...,
       [-1.7219838 , -1.8906404 ,  3.097601  , -0.4756199 ],
       [-2.2620668 , -2.6661584 ,  2.0910578 ,  1.7683979 ],
       [-1.6994187 , -1.9174868 ,  3.4385614 , -0.95996207]],
      dtype=float32)

In [56]:
accuracy_per_class(predictions, true_vals)

Class: Cannot Say
Accuracy: 0/17

Class: Negative
Accuracy: 21/60

Class: Postive
Accuracy: 446/565

Class: No Sentiment
Accuracy: 214/313



# Inference

In [64]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

# Inference

In [68]:
df_infer = pd.read_csv("Test.csv")
df_infer.set_index("Text_ID", inplace=True)

In [69]:
df_infer.head()

,Product_Description,Product_Type
Text_ID,,
5786,RT @mention Going to #SXSW? The new iPhone gui...,7
5363,RT @mention 95% of iPhone and Droid apps have ...,9
6716,RT @mention Thank you to @mention for letting ...,9
4339,#Thanks @mention we're lovin' the @mention app...,7
66,At #sxsw? @mention / @mention wanna buy you a ...,9


In [194]:
df_infer.shape

(2728, 2)

In [91]:
df_infer.Product_Description

Text_ID
5786    RT @mention Going to #SXSW? The new iPhone gui...
5363    RT @mention 95% of iPhone and Droid apps have ...
6716    RT @mention Thank you to @mention for letting ...
4339    #Thanks @mention we're lovin' the @mention app...
66      At #sxsw? @mention / @mention wanna buy you a ...
                              ...                        
5705    RT @mention eww and LOL. RT @mention Just saw ...
7556    Free 22 track #sxsw sampler album on iTunes. #...
7302    Setting up for the Google #gsdm  #sxsw party. ...
5286    RT @mention #SXSW Come see Bitbop in Austin #g...
7047    So many Google products. isn't it time to  tra...
Name: Product_Description, Length: 2728, dtype: object

In [195]:
encoded_data_infer = tokenizer.batch_encode_plus(
    df_infer.Product_Description.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [196]:
input_ids_inf = encoded_data_infer['input_ids']
attention_masks_inf = encoded_data_infer['attention_mask']
#labels_val = torch.tensor(df[df.data_type=='val'].Sentiment.values)

In [197]:
dataset_inf = TensorDataset(input_ids_inf, attention_masks_inf)

In [198]:
dataloader_inf = DataLoader(dataset_inf, 
                                   sampler=SequentialSampler(dataset_inf), 
                                   batch_size=batch_size)

In [182]:
# to check the infere -
'''
for batch in dataloader_validation:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
'''
            

In [183]:
outputs

(tensor(1.2956, device='cuda:0'), tensor([[-1.7272, -2.4389,  1.4464,  3.1177],
         [-1.7102, -1.6756,  3.0056, -0.6153],
         [-1.2166, -1.6393, -0.1042,  3.6617],
         [-0.8531, -0.2940,  2.1861, -1.6407],
         [-1.4026, -1.2600,  2.5940, -1.4467],
         [-1.2078, -1.4049,  2.4812, -0.5820],
         [-1.2861, -0.5599, -0.0201,  2.6236],
         [-2.1996, -1.7122,  1.3483,  1.3861],
         [-0.7546, -1.1481, -0.9170,  3.7375],
         [-1.8484, -2.4207,  3.7079, -0.4236],
         [-1.2453, -2.0495,  2.3225,  2.1280],
         [-1.8706, -2.5760,  3.4178,  1.5196],
         [-0.5785,  1.3717,  0.6026, -1.8252],
         [-1.0261, -0.7825,  2.8912, -2.2829],
         [-1.9756, -2.5297,  3.2120,  1.1635],
         [-2.1066, -2.6180,  2.4288,  1.1728],
         [-1.3901, -1.1649,  2.9694, -1.8766],
         [-0.5405, -1.2300, -1.1709,  3.7606],
         [-0.8982, -0.1106, -0.5827,  2.0463],
         [-0.9395, -1.5990, -0.4378,  3.7011],
         [-1.3397, -1.4029,

In [186]:
def infer(dataloader_inf):

    model.eval()
    
    loss_val_total = 0
    predictions = []
    
    for batch in dataloader_inf:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],      
                  'labels' :None,           
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        #loss = outputs[0]
        logits = outputs[0]
        #loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        #label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        #true_vals.append(label_ids)
    
    #loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    #true_vals = np.concatenate(true_vals, axis=0)
            
    return predictions

In [199]:
prds = infer(dataloader_inf)

In [200]:
prds

array([[-1.8178201 , -2.3899753 ,  1.4717472 ,  3.1546328 ],
       [-0.39357972,  2.0907583 , -0.30460644, -1.6036015 ],
       [-2.2310119 , -1.8959662 ,  1.8798711 ,  2.6908827 ],
       ...,
       [-2.0794573 , -2.7258637 ,  2.8214118 ,  0.6853338 ],
       [-1.476132  , -1.8144056 ,  3.6256964 , -1.4940987 ],
       [-1.5218972 , -1.3372592 ,  3.4206662 , -1.3072776 ]],
      dtype=float32)

In [212]:
from scipy.special import softmax

probabilities = softmax(prds, axis=1)

In [213]:
probabilities

array([[0.00578784, 0.00326612, 0.15529487, 0.8356512 ],
       [0.06951965, 0.83376163, 0.07598855, 0.02073012],
       [0.00498353, 0.00696699, 0.30399808, 0.6840513 ],
       ...,
       [0.00658734, 0.00345128, 0.8853825 , 0.10457888],
       [0.0059874 , 0.00426902, 0.9838629 , 0.00588079],
       [0.00696519, 0.00837761, 0.9760246 , 0.00863259]], dtype=float32)

In [220]:
def get_pred(x):
    if x >= 0.5:
        return 1
    return 0

predictions_prob = [[get_pred(probabilities) for probabilities in row] for row in probabilities]

In [221]:
predictions_prob

[[0, 0, 0, 1],
 [0, 1, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 1],
 [0, 1, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 1, 0, 0],
 [0, 1, 0, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 1, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 0, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 1,

In [222]:
df_submission = pd.DataFrame(predictions_prob,columns=['Class_0','Class_1','Class_2','Class_3'])

In [227]:
df_submission.to_csv("Submisson.csv",index=False)

In [218]:
np.around(a,2)

array([[0.01, 0.  , 0.16, 0.84],
       [0.07, 0.83, 0.08, 0.02],
       [0.  , 0.01, 0.3 , 0.68],
       ...,
       [0.01, 0.  , 0.89, 0.1 ],
       [0.01, 0.  , 0.98, 0.01],
       [0.01, 0.01, 0.98, 0.01]], dtype=float32)

In [219]:
np.floor(a)

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)

# create submisison file 

In [217]:
import numpy
a = numpy.asarray(predictions_prob)
#b = numpy.around(a)
numpy.savetxt("Submisson_new.csv", b, delimiter=",")

In [206]:
prds.shape

(2728, 4)

In [207]:
preds_f = np.argmax(prds,)

In [211]:
preds_f

2274

In [201]:
preds_flat = np.argmax(prds, axis=1).flatten()

In [202]:
preds_flat

array([3, 1, 3, ..., 2, 2, 2])

In [203]:
len(preds_flat)

2728

In [204]:
preds_flat.shape

(2728,)